In [42]:
import yfinance as yf
import pandas as pd
import warnings
import plotly.express as px

from optimisation_markowitz import MarkowitzOptimization
from optimisation_simm import SingleFactorModel, SingleFactorModelOptimization
from utils import Utils
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
start_date = "2023-04-01"
end_date = "2024-04-01"
annualized_factor = 252

We will use S&P500 as benchmark to evaluate the performance with Markowitz Optimization and Single Index Market Model. The return over the last year will be used as the market return in the formula.

In [44]:
# Get close data for S&P 500 Index
market_index_data = Utils.get_historical_data("^SPX", start_date, end_date)

# calculate simple return for market index
market_index_simple_return = Utils.calculate_simple_return(market_index_data)

# calculate daily market data returns.
# use that to calculate expected returns and variance
market_index_daily_returns = Utils.calculate_daily_returns(market_index_data)
market_index_data_expected_return = market_index_daily_returns.mean()
market_index_data_variance = market_index_daily_returns.var()

print(f"Market Index Simple Return from {start_date} - {end_date} in %: ", market_index_simple_return * 100)
print("Market Index Expected daily Return in %: ", market_index_data_expected_return * 100)
print("Market Index daily Variance in %: ", market_index_data_variance * 100)

[*********************100%%**********************]  1 of 1 completed
Market Index Simple Return from 2023-04-01 - 2024-04-01 in %:  27.393324206617265
Market Index Expected daily Return in %:  0.10028595199917367
Market Index daily Variance in %:  0.005254458483042907


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [45]:
risk_free_rate_df = yf.download("^IRX", start_date, end_date)
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

# Combined Sector ETFs

Following our analysis for Technology, HealthCare and Financials sector, we will combine the 2 highest performing stocks from each sector in an effort to create a more diverse portfolio. The following stocks are included:
1. VanEck SemiConductor (SMH) -> Tech
2. iShares Semiconductor (SOXX) -> Tech
3. iShares US Financials ETF (IYF) -> Financials
4. Vanguard Financials ETF (VFH)  -> Financials
5. SPDR S&P Biotech ETF (XBI) -> HealthCare
6. Healthcare Sector SPDT Fund (XLV) -> HealthCare 

In [46]:
# get the data
smh_data = Utils.get_historical_data("SMH", start_date, end_date)
soxx_data = Utils.get_historical_data("SOXX", start_date, end_date)
iyf_data = Utils.get_historical_data("IYF", start_date, end_date)
vfh_data = Utils.get_historical_data("VFH", start_date, end_date)
xbi_data = Utils.get_historical_data("XBI", start_date, end_date)
xlv_data = Utils.get_historical_data("XLV", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [47]:

smh_simple_return = Utils.calculate_simple_return(smh_data)
soxx_simple_return = Utils.calculate_simple_return(soxx_data)
iyf_simple_return = Utils.calculate_simple_return(iyf_data)
vfh_simple_return = Utils.calculate_simple_return(vfh_data)
xbi_simple_return = Utils.calculate_simple_return(xbi_data)
xlv_simple_return = Utils.calculate_simple_return(xlv_data)

simple_returns_dict = {
    "SMH": smh_simple_return,
    "SOXX": soxx_simple_return,
    "IYF": iyf_simple_return,
    "VFH": vfh_simple_return,
    "XBI": xbi_simple_return,
    "XLV": xlv_simple_return
}

for ticker, simple_return in simple_returns_dict.items():
    print(f"{ticker} -> Simple Return from {start_date} - {end_date} in %: ", simple_return * 100)

SMH -> Simple Return from 2023-04-01 - 2024-04-01 in %:  72.98801691288277
SOXX -> Simple Return from 2023-04-01 - 2024-04-01 in %:  57.11977609580361
IYF -> Simple Return from 2023-04-01 - 2024-04-01 in %:  36.68568980169895
VFH -> Simple Return from 2023-04-01 - 2024-04-01 in %:  34.56978857828605
XBI -> Simple Return from 2023-04-01 - 2024-04-01 in %:  23.162156584619463
XLV -> Simple Return from 2023-04-01 - 2024-04-01 in %:  14.699943785182402


In [81]:
# plot the simple retruns using plotly for all stocks along with market index
fig = px.line(title="Simple Returns for Combined Sector ETFs and Market Index")
fig.add_scatter(x=smh_data.index, y=smh_data['simple_return'], mode='lines', name='SMH')
fig.add_scatter(x=soxx_data.index, y=soxx_data['simple_return'], mode='lines', name='SOXX')
fig.add_scatter(x=iyf_data.index, y=iyf_data['simple_return'], mode='lines', name='IYF')
fig.add_scatter(x=vfh_data.index, y=vfh_data['simple_return'], mode='lines', name='VFH')
fig.add_scatter(x=xbi_data.index, y=xbi_data['simple_return'], mode='lines', name='XBI')
fig.add_scatter(x=xlv_data.index, y=xlv_data['simple_return'], mode='lines', name='XLV')

fig.add_scatter(x=market_index_data.index, y=market_index_data['simple_return'], mode='lines', name='Market Index')
fig.show()

On observing the simple returns, we see that both ETFs from Healthcare sector underperformed relative to the market.

In [49]:
smh_daily_returns = Utils.calculate_daily_returns(smh_data)
soxx_daily_returns = Utils.calculate_daily_returns(soxx_data)
iyf_daily_returns = Utils.calculate_daily_returns(iyf_data)
vfh_daily_returns = Utils.calculate_daily_returns(vfh_data)
xbi_daily_returns = Utils.calculate_daily_returns(xbi_data)
xlv_daily_returns = Utils.calculate_daily_returns(xlv_data)


daily_returns_df = pd.DataFrame([smh_daily_returns, soxx_daily_returns, iyf_daily_returns, vfh_daily_returns, xbi_daily_returns, xlv_daily_returns]).T
daily_returns_df.columns = ["SMH", "SOXX", "IYF", "VFH", "XBI", "XLV"]

In [50]:
cov_matrix = daily_returns_df.cov()
cov_matrix

,SMH,SOXX,IYF,VFH,XBI,XLV
SMH,0.000298,0.000294,0.000061,0.000066,0.000073,0.000018
SOXX,0.000294,0.000304,0.000066,0.000071,0.000092,0.000019
IYF,0.000061,0.000066,0.000078,0.000077,0.000070,0.000028
VFH,0.000066,0.000071,0.000077,0.000078,0.000075,0.000029
XBI,0.000073,0.000092,0.000070,0.000075,0.000325,0.000055
XLV,0.000018,0.000019,0.000028,0.000029,0.000055,0.000044


In [51]:
corr_matrix = daily_returns_df.corr()
corr_matrix

,SMH,SOXX,IYF,VFH,XBI,XLV
SMH,1.000000,0.975605,0.402219,0.432813,0.233002,0.156475
SOXX,0.975605,1.000000,0.427095,0.458887,0.292228,0.162786
IYF,0.402219,0.427095,1.000000,0.986828,0.437255,0.475299
VFH,0.432813,0.458887,0.986828,1.000000,0.472433,0.491187
XBI,0.233002,0.292228,0.437255,0.472433,1.000000,0.459904
XLV,0.156475,0.162786,0.475299,0.491187,0.459904,1.000000


In [80]:
# plot correlation using plotly with cell values and centre aligned title
fig = px.imshow(corr_matrix, color_continuous_scale='RdBu', title="Correlation Matrix between returns of Combined Sector ETFs")
fig.update_layout(title_x=0.5)
fig.show()

As expected, we observe that correlation among ETFs in the same sector is high whereas the correlation between ETFs in different sectors is low

In [52]:
# standard deviations between daily returns
std_dev = daily_returns_df.std()
std_dev

SMH     0.017272
SOXX    0.017427
IYF     0.008816
VFH     0.008851
XBI     0.018034
XLV     0.006632
dtype: float64

In [53]:
# daily mean return
mean_daily_return = daily_returns_df.mean()
mean_daily_return

SMH     0.002359
SOXX    0.001974
IYF     0.001300
VFH     0.001237
XBI     0.001002
XLV     0.000575
dtype: float64

## Optimal tangency portfolio using Markowitz model 

In [54]:
# Run the Markowitz Portfolio Optimization
simple_returns_list = list(simple_returns_dict.values())
markowitz = MarkowitzOptimization(simple_returns_list, cov_matrix, risk_free_rate)


In [55]:
# find the weights of the portfolio with short shelling
markowitz_weights_with_ss = markowitz.find_tangency_portfolio_with_short_selling()

In [56]:
# create the wieghts dictionary
markowitz_weights_with_ss_dict = {
    "SMH": markowitz_weights_with_ss[0],
    "SOXX": markowitz_weights_with_ss[1],
    "IYF": markowitz_weights_with_ss[2],
    "VFH": markowitz_weights_with_ss[3],
    "XBI": markowitz_weights_with_ss[4],
    "XLV": markowitz_weights_with_ss[5]
}

for ticker, weight in markowitz_weights_with_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights with Short Selling in %: ", weight * 100)

SMH -> Markowitz Weights with Short Selling in %:  287.8216325311891
SOXX -> Markowitz Weights with Short Selling in %:  -253.69248072252876
IYF -> Markowitz Weights with Short Selling in %:  347.0757090688817
VFH -> Markowitz Weights with Short Selling in %:  -281.13872072991404
XBI -> Markowitz Weights with Short Selling in %:  12.516901411167666
XLV -> Markowitz Weights with Short Selling in %:  -12.583041558795655


In [57]:
# find the weights of the portfolio without short shelling
markowitz_weights_without_ss = markowitz.find_tangency_portfolio_no_short_selling()

In [58]:
# weights dictionary
markowitz_weights_without_ss_dict = {
    "SMH": markowitz_weights_without_ss[0],
    "SOXX": markowitz_weights_without_ss[1],
    "IYF": markowitz_weights_without_ss[2],
    "VFH": markowitz_weights_without_ss[3],
    "XBI": markowitz_weights_without_ss[4],
    "XLV": markowitz_weights_without_ss[5]
}

for ticker, weight in markowitz_weights_without_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling in %: ", weight * 100)

SMH -> Markowitz Weights without Short Selling in %:  38.935314910716464
SOXX -> Markowitz Weights without Short Selling in %:  2.3190173739640052e-11
IYF -> Markowitz Weights without Short Selling in %:  61.06468508926167
VFH -> Markowitz Weights without Short Selling in %:  0.0
XBI -> Markowitz Weights without Short Selling in %:  3.1190761778931986e-12
XLV -> Markowitz Weights without Short Selling in %:  0.0


In [59]:
expected_portfolio_returns_with_ss, volatility_with_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_with_ss, simple_returns_list, cov_matrix)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)
print("Expected Portfolio Returns with Short Selling in %: ", expected_portfolio_returns_with_ss * 100)

Volatility with Short Selling in %:  1.4059409639209441
Expected Portfolio Returns with Short Selling in %:  96.3542657770171


In [60]:
# calculate sharpe ratio
portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return with Short Selling in %: ", portfolio_mean_return_with_ss * 100)

sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss, volatility_with_ss, risk_free_rate)
print("Sharpe Ratio with Short Selling: ", sharpe_ratio_with_ss)

Mean Daily Portfolio Return with Short Selling in %:  4.555329573777693
Sharpe Ratio with Short Selling:  0.6053024348056754


In [61]:
expected_portfolio_return_without_ss, volatility_without_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss, simple_returns_list, cov_matrix)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)
print("Expected Portfolio Returns without Short Selling in %: ", expected_portfolio_return_without_ss * 100)

Volatility without Short Selling in %:  1.0164977243300923
Expected Portfolio Returns without Short Selling in %:  50.82011518236271


In [62]:
# calculate sharpe ratio
portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return without Short Selling in %: ", portfolio_mean_return_without_ss * 100)

sharpe_ratio_without_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_without_ss, volatility_without_ss, risk_free_rate)
print("Sharpe Ratio without Short Selling: ", sharpe_ratio_without_ss)

Mean Daily Portfolio Return without Short Selling in %:  2.7181473245233367
Sharpe Ratio without Short Selling:  0.32705161978404607


We also observe that without weights constraint in optimization without short selling, SMH and IYF account for around 99% proportion in the portfolio. To promote diversification , we will add a constraint that allows a maximum of 25% in each ETF

In [63]:
# we are interested in finding optimal weights when an asset can hold 30% of the portfolio max
markowitz_weights_without_ss_pct_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.25)

In [64]:
markowitz_weights_dict_without_ss_pct_constraint = {
    "SMH": markowitz_weights_without_ss_pct_constraint[0],
    "SOXX": markowitz_weights_without_ss_pct_constraint[1],
    "IYF": markowitz_weights_without_ss_pct_constraint[2],
    "VFH": markowitz_weights_without_ss_pct_constraint[3],
    "XBI": markowitz_weights_without_ss_pct_constraint[4],
    "XLV": markowitz_weights_without_ss_pct_constraint[5]
}

for ticker, weight in markowitz_weights_dict_without_ss_pct_constraint.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling and 30% constraint in %: ", weight * 100)

SMH -> Markowitz Weights without Short Selling and 30% constraint in %:  25.000000000003926
SOXX -> Markowitz Weights without Short Selling and 30% constraint in %:  4.1907440295751996e-07
IYF -> Markowitz Weights without Short Selling and 30% constraint in %:  25.000000000013205
VFH -> Markowitz Weights without Short Selling and 30% constraint in %:  25.000000000008278
XBI -> Markowitz Weights without Short Selling and 30% constraint in %:  0.0
XLV -> Markowitz Weights without Short Selling and 30% constraint in %:  25.000000000003595


In [65]:
expected_returns_without_ss_with_constraint, volatility_without_ss_with_constraint = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss_pct_constraint, simple_returns_list, cov_matrix)
print("Volatility with Short Selling and Constraint in %: ", volatility_without_ss_with_constraint * 100)
print("Expected Portfolio Returns with Short Selling and Constraint in %: ", expected_returns_without_ss_with_constraint * 100)

Volatility with Short Selling and Constraint in %:  0.8125092672359538
Expected Portfolio Returns with Short Selling and Constraint in %:  39.73586000889801


In [66]:
# find the Sharpe Ratio
portfolio_mean_return_with_ss_with_constraint = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss_pct_constraint, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return with Short Selling and Constraint in %: ", portfolio_mean_return_with_ss_with_constraint * 100)

sharpe_no_ss_with_constraint = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss_with_constraint, volatility_without_ss_with_constraint, risk_free_rate)
print("Sharpe Ratio with Short Selling and Constraint: ", sharpe_no_ss_with_constraint)

Mean Daily Portfolio Return with Short Selling and Constraint in %:  2.1712673496404067
Sharpe Ratio with Short Selling and Constraint:  0.26114896491885004


## Single Index Market Model

We will now compare results from markowitz model with single index market model. We will use S&P500 Index as benchmark to evaluate the performance with Single Index Market Model.

From this point, we will run `SingleFactorModel` class over all ETFs calculating beta, alpha and tau. We will then calculate the expected return using the formula:

$$
u_i = \alpha_i + \beta_i \cdot u_m

In [67]:
def get_simm_params(asset_return: pd.DataFrame, market_return: pd.DataFrame):
  model = SingleFactorModel(asset_return, market_return)
  model.fit()
  
  params = model.get_params()
  alpha = params.get("alpha").get("value")
  beta = params.get("beta").get("value")
  tau = params.get("tau").get("value")
  
  return (alpha, beta, tau)
  

In [68]:
# run SIMM for params
smh_alpha, smh_beta, smh_tau = get_simm_params(smh_daily_returns, market_index_daily_returns)
soxx_alpha, soxx_beta, soxx_tau = get_simm_params(soxx_daily_returns, market_index_daily_returns)
iyf_alpha, iyf_beta, iyf_tau = get_simm_params(iyf_daily_returns, market_index_daily_returns)
vfh_alpha, vfh_beta, vfh_tau = get_simm_params(vfh_daily_returns, market_index_daily_returns)
xbi_alpha, xbi_beta, xbi_tau = get_simm_params(xbi_daily_returns, market_index_daily_returns)
xlv_alpha, xlv_beta, xlv_tau = get_simm_params(xlv_daily_returns, market_index_daily_returns)

In [69]:
# get expected returns using alpha, beta and expected market return
smh_expected_return = smh_alpha + (smh_beta * market_index_data_expected_return)
soxx_expected_return = soxx_alpha + (soxx_beta * market_index_data_expected_return)
iyf_expected_return = iyf_alpha + (iyf_beta * market_index_data_expected_return)
vfh_expected_return = vfh_alpha + (vfh_beta * market_index_data_expected_return)
xbi_expected_return = xbi_alpha + (xbi_beta * market_index_data_expected_return)
xlv_expected_return = xlv_alpha + (xlv_beta * market_index_data_expected_return)

In [70]:
# we can now pack the expected returns into a dictionary along with beta and tau
expected_returns_dict = {
    "SMH": smh_expected_return,
    "SOXX": soxx_expected_return,
    "IYF": iyf_expected_return,
    "VFH": vfh_expected_return,
    "XBI": xbi_expected_return,
    "XLV": xlv_expected_return
}

alpha_dict = {
    "SMH": smh_alpha,
    "SOXX": soxx_alpha,
    "IYF": iyf_alpha,
    "VFH": vfh_alpha,
    "XBI": xbi_alpha,
    "XLV": xlv_alpha
}

beta_dict = {
    "SMH": smh_beta,
    "SOXX": soxx_beta,
    "IYF": iyf_beta,
    "VFH": vfh_beta,
    "XBI": xbi_beta,
    "XLV": xlv_beta
}

tau_dict = {
    "SMH": smh_tau,
    "SOXX": soxx_tau,
    "IYF": iyf_tau,
    "VFH": vfh_tau,
    "XBI": xbi_tau,
    "XLV": xlv_tau
}

for i in range(len(expected_returns_dict.keys())):
  print(f"{ list(expected_returns_dict.keys())[i] } -> Expected Daily Return in %: ", list(expected_returns_dict.values())[i] * 100)
  print(f"{ list(alpha_dict.keys())[i] } -> Alpha: ", list(alpha_dict.values())[i])
  print(f"{ list(beta_dict.keys())[i] } -> Beta: ", list(beta_dict.values())[i])
  print(f"{ list(tau_dict.keys())[i] } -> Tau: ", list(tau_dict.values())[i])
  print()

SMH -> Expected Daily Return in %:  0.2359472140425166
SMH -> Alpha:  0.0005917930464843844
SMH -> Beta:  1.762638793073776
SMH -> Tau:  0.011645399970193673

SOXX -> Expected Daily Return in %:  0.19738984714288635
SOXX -> Alpha:  0.00020463506937442246
SOXX -> Beta:  1.7642185837444302
SOXX -> Tau:  0.01186330148883692

IYF -> Expected Daily Return in %:  0.12996119757147157
IYF -> Alpha:  0.0004063678366900972
IYF -> Beta:  0.8906971726528343
IYF -> Tau:  0.006014964620947862

VFH -> Expected Daily Return in %:  0.12369351663862894
VFH -> Alpha:  0.00029383932578478246
VFH -> Beta:  0.9404067287602534
VFH -> Tau:  0.005657369805698939

XBI -> Expected Daily Return in %:  0.10016452539732086
XBI -> Alpha:  -6.0108421295658964e-05
XBI -> Beta:  1.0587262264585335
XBI -> Tau:  0.01635272821436717

XLV -> Expected Daily Return in %:  0.057506014817705085
XLV -> Alpha:  4.5739269690922064e-05
XLV -> Beta:  0.5278115906906784
XLV -> Tau:  0.005427681831477485



In [71]:
# run the SIMM optimization
expected_returns_list = list(expected_returns_dict.values())
beta_list = list(beta_dict.values())
tau_list = list(tau_dict.values())
alpha_list = list(alpha_dict.values())

simm_optimization = SingleFactorModelOptimization(expected_returns_list, beta_list, tau_list, market_index_data_variance, risk_free_rate)

In [72]:
# find the optimal weights when short selling is allowed
simm_weights_with_ss = simm_optimization.find_weights_short_selling_allowed()

In [73]:
simm_weights_with_ss_dict = {
    "SMH": simm_weights_with_ss[0],
    "SOXX": simm_weights_with_ss[1],
    "IYF": simm_weights_with_ss[2],
    "VFH": simm_weights_with_ss[3],
    "XBI": simm_weights_with_ss[4],
    "XLV": simm_weights_with_ss[5]
}

for ticker, weight in simm_weights_with_ss_dict.items():
    print(f"{ticker} -> SIMM Weights with Short Selling in %: ", weight * 100)

SMH -> SIMM Weights with Short Selling in %:  -10.199520730551205
SOXX -> SIMM Weights with Short Selling in %:  -9.68115568206959
IYF -> SIMM Weights with Short Selling in %:  26.26522319469917
VFH -> SIMM Weights with Short Selling in %:  25.781525068999755
XBI -> SIMM Weights with Short Selling in %:  1.992836018748546
XLV -> SIMM Weights with Short Selling in %:  65.84109213017332


With weights found, we can find the expected return of portfolio and the risk using the formula:

$$
u_p = \alpha_p + \beta_p \cdot u_m

\newline

\sigma_p^2 = \beta_p^2 \cdot \sigma_m^2 + \tau_p^2

\newline

where:
\newline

\alpha_p = \sum_{i=1}^{n} x_i \cdot \alpha_i

\newline

\beta_p = \sum_{i=1}^{n} x_i \cdot \beta_i

\newline

\tau_p^2 = \sum_{i=1}^{n} x_i^2 \cdot \tau_i^2

In [74]:
simm_expected_return_with_ss, simm_portfolio_risk_with_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_with_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk with Short Selling in %: ", simm_portfolio_risk_with_ss * 100)
print("Portfolio Expected Return with Short Selling in %: ", simm_expected_return_with_ss * 100)

Portfolio Risk with Short Selling in %:  0.5982888488598128
Portfolio Expected Return with Short Selling in %:  49.44330650358432


In [75]:
# find the sharpe ratio
simm_portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(simm_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Daily  Portfolio Return with Short Selling in %: ", simm_portfolio_mean_return_with_ss * 100)

simm_sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_with_ss, simm_portfolio_risk_with_ss, risk_free_rate)
print("SIMM Sharpe Ratio with Short Selling: ", simm_sharpe_ratio_with_ss)

Mean Daily  Portfolio Return with Short Selling in %:  0.9954629751577484
SIMM Sharpe Ratio with Short Selling:  0.07633559889609569


In [76]:
# find optimal weights when short selling is not allowed
simm_weights_no_ss = simm_optimization.find_weights_no_short_selling()

In [77]:
simm_weights_no_ss_dict = {
    "SMH": simm_weights_no_ss[0],
    "SOXX": simm_weights_no_ss[1],
    "IYF": simm_weights_no_ss[2],
    "VFH": simm_weights_no_ss[3],
    "XBI": simm_weights_no_ss[4],
    "XLV": simm_weights_no_ss[5]
}

for ticker, weight in simm_weights_no_ss_dict.items():
  print(f"{ticker} -> SIMM Weights without Short Selling in %: ", weight * 100)
    

SMH -> SIMM Weights without Short Selling in %:  12.81709853908496
SOXX -> SIMM Weights without Short Selling in %:  12.361648269721211
IYF -> SIMM Weights without Short Selling in %:  24.276456997065534
VFH -> SIMM Weights without Short Selling in %:  28.974062443369192
XBI -> SIMM Weights without Short Selling in %:  3.9041709802185185
XLV -> SIMM Weights without Short Selling in %:  17.66656277054059


In [78]:
# we will now work to find the expected return and risk of the portfolio
simm_expected_return_without_ss, simm_portfolio_risk_no_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_no_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk without Short Selling in %: ", simm_portfolio_risk_no_ss * 100)
print("Expected Portfolio Return without Short Selling in %: ", simm_expected_return_without_ss * 100)

Portfolio Risk without Short Selling in %:  0.4755000032004905
Expected Portfolio Return without Short Selling in %:  106.72903355075117


In [79]:
# find the sharpe ratio
simm_portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(simm_weights_no_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return without Short Selling in %: ", simm_portfolio_mean_return_without_ss * 100)

simm_sharpe_ratio_no_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_without_ss, simm_portfolio_risk_no_ss, risk_free_rate)
print("SIMM Sharpe Ratio without Short Selling: ", simm_sharpe_ratio_no_ss)

Mean Daily Portfolio Return without Short Selling in %:  2.1605390473760813
SIMM Sharpe Ratio without Short Selling:  0.32385912395501626
